In [1]:
import sagemaker
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Setup session
session = sagemaker.Session()
bucket = "customer-churn-data-michael"  # replace with your bucket name
prefix = "churn"

# Download dataset from S3
s3 = boto3.client("s3")
s3.download_file(bucket, "WA_Fn-UseC_-Telco-Customer-Churn.csv.csv", "churn.csv")

# Load locally
df = pd.read_csv("churn.csv")

# Encode label
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

# Train-test split
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Save CSVs for training
train.to_csv("train.csv", index=False, header=False)
test.to_csv("test.csv", index=False, header=False)

# Upload back to S3
train_path = session.upload_data("train.csv", bucket=bucket, key_prefix=prefix)
test_path = session.upload_data("test.csv", bucket=bucket, key_prefix=prefix)

print("Train path:", train_path)
print("Test path:", test_path)


Train path: s3://customer-churn-data-michael/churn/train.csv
Test path: s3://customer-churn-data-michael/churn/test.csv


In [3]:
import sagemaker
from sagemaker import image_uris

# Get XGBoost image URI
container = image_uris.retrieve("xgboost", boto3.Session().region_name, "1.5-1")

# Set up estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=f"s3://{bucket}/{prefix}/output",
    sagemaker_session=session
)

# Set hyperparameters
xgb.set_hyperparameters(
    objective="binary:logistic",
    num_round=100,
    max_depth=5,
    eta=0.2,
    subsample=0.8,
    eval_metric="auc"
)

# Define training data inputs
train_input = sagemaker.inputs.TrainingInput(
    s3_data=train_path,
    content_type="csv"
)
test_input = sagemaker.inputs.TrainingInput(
    s3_data=test_path,
    content_type="csv"
)

# Launch training
xgb.fit({"train": train_input, "validation": test_input})


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-09-17-20-53-40-469
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:38                                                                                   │
│                                                                                                  │
│   35 )                                                                                           │
│   36                                                                                             │
│   37 # Launch training                                                                           │
│ ❱ 38 xgb.fit({"train": train_input, "validation": test_input})                                   │
│   39                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:168 in wrapper                                                                       │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:139 in wrapper                                                                       │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                   

In [1]:
# === Day 2: Train XGBoost (low-cost + quota-safe) ===
import os, boto3, sagemaker
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput
from sklearn.model_selection import train_test_split
import pandas as pd

# ---- Config (edit only these if needed) ----
# Source data (your existing bucket with the CSV)
SRC_BUCKET   = "customer-churn-data-michael"     # your bucket with the raw CSV
SRC_KEY      = "WA_Fn-UseC_-Telco-Customer-Churn.csv"  # exact key in that bucket

# Where training inputs/outputs will live (same region as notebook/training)
# Use the default SageMaker bucket in this region to avoid cross-region issues
session = sagemaker.Session()
REGION  = session.boto_region_name
ROLE    = sagemaker.get_execution_role()
DST_BUCKET = session.default_bucket()
PREFIX     = "churn"   # folder/prefix in the bucket

print(f"Region: {REGION}")
print(f"Using SageMaker bucket: s3://{DST_BUCKET}/{PREFIX}")

# ---- 1) Download CSV locally from your source S3 ----
s3 = boto3.client("s3")
local_csv = "churn.csv"
s3.download_file(SRC_BUCKET, SRC_KEY, local_csv)
print("Downloaded:", local_csv)

# ---- 2) Load, prep, split ----
df = pd.read_csv(local_csv)
# encode label
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_csv = "train.csv"
test_csv  = "test.csv"
train_df.to_csv(train_csv, index=False, header=False)  # XGBoost expects no header
test_df.to_csv(test_csv, index=False, header=False)

# ---- 3) Upload to SageMaker bucket in this region ----
train_s3 = session.upload_data(train_csv, bucket=DST_BUCKET, key_prefix=f"{PREFIX}/train")
test_s3  = session.upload_data(test_csv,  bucket=DST_BUCKET, key_prefix=f"{PREFIX}/test")
print("Train S3:", train_s3)
print("Test  S3:", test_s3)

# ---- 4) Build TrainingInput channels ----
train_input = TrainingInput(train_s3,  content_type="text/csv")
test_input  = TrainingInput(test_s3,   content_type="text/csv")

# ---- 5) Get the XGBoost container for this region ----
# Pick a stable, low-cost CPU version of XGBoost
container_image = image_uris.retrieve(framework="xgboost", region=REGION, version="1.7-1")

# ---- 6) Try small/cheap instance types until one is allowed by your quotas ----
candidate_types = ["ml.m5.large", "ml.c5.large", "ml.m4.xlarge"]  # cheapest first

last_err = None
for itype in candidate_types:
    try:
        print(f"\nTrying training instance_type = {itype}")
        xgb = sagemaker.estimator.Estimator(
            image_uri=container_image,
            role=ROLE,
            instance_count=1,
            instance_type=itype,
            output_path=f"s3://{DST_BUCKET}/{PREFIX}/output",
            sagemaker_session=session,
        )
        # Modest hyperparams to keep job short/cheap
        xgb.set_hyperparameters(
            objective="binary:logistic",
            num_round=80,
            max_depth=5,
            eta=0.2,
            subsample=0.8,
            eval_metric="auc",
        )

        # Launch training (this automatically shuts down the training instance when done)
        xgb.fit({"train": train_input, "validation": test_input})
        print("\n✅ Training started successfully on", itype)
        last_err = None
        break
    except Exception as e:
        msg = str(e)
        last_err = e
        if "ResourceLimitExceeded" in msg or "limit" in msg.lower():
            print(f"⚠️  No quota for {itype}. Trying next option…")
        else:
            # Any other error should surface immediately
            raise

if last_err:
    raise last_err

# After the job completes, the trained model artifact is at:
# s3://{DST_BUCKET}/{PREFIX}/output/<training-job-name>/output/model.tar.gz


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Region: us-east-1
Using SageMaker bucket: s3://sagemaker-us-east-1-191687650900/churn


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:27                                                                                   │
│                                                                                                  │
│   24 # ---- 1) Download CSV locally from your source S3 ----                                     │
│   25 s3 = boto3.client("s3")                                                                     │
│   26 local_csv = "churn.csv"                                                                     │
│ ❱ 27 s3.download_file(SRC_BUCKET, SRC_KEY, local_csv)                                            │
│   28 print("Downloaded:", local_csv)                                                             │
│   29                                                                                             │
│   30 # ---- 2) Load, prep, split ----                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/boto3/s3/inject.py:223 in     │
│ download_file                                                                                    │
│                                                                                                  │
│   220 │   │   transfer.                                                                          │
│   221 │   """                                                                                    │
│   222 │   with S3Transfer(self, Config) as transfer:                                             │
│ ❱ 223 │   │   return transfer.download_file(                                                     │
│   224 │   │   │   bucket=Bucket,                                                                 │
│   225 │   │   │   key=Key,                                                                       │
│   226 │   │   │   filename=Filename,                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/boto3/s3/transfer.py:406 in   │
│ download_file                                                                                    │
│                                                                                                  │
│   403 │   │   │   bucket, key, filename, extra_args, subscribers                                 │
│   404 │   │   )                                                                                  │
│   405 │   │   try:                                                                               │
│ ❱ 406 │   │   │   future.result()                          

In [2]:
# === Day 2: Train XGBoost (low-cost + quota-safe) ===
import os, boto3, sagemaker
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput
from sklearn.model_selection import train_test_split
import pandas as pd

# ---- Config (edit only these if needed) ----
# Source data (your existing bucket with the CSV)
SRC_BUCKET   = "customer-churn-data-michael"     # your bucket with the raw CSV
SRC_KEY      = "WA_Fn-UseC_-Telco-Customer-Churn.csv.csv"  # exact key in that bucket

# Where training inputs/outputs will live (same region as notebook/training)
# Use the default SageMaker bucket in this region to avoid cross-region issues
session = sagemaker.Session()
REGION  = session.boto_region_name
ROLE    = sagemaker.get_execution_role()
DST_BUCKET = session.default_bucket()
PREFIX     = "churn"   # folder/prefix in the bucket

print(f"Region: {REGION}")
print(f"Using SageMaker bucket: s3://{DST_BUCKET}/{PREFIX}")

# ---- 1) Download CSV locally from your source S3 ----
s3 = boto3.client("s3")
local_csv = "churn.csv"
s3.download_file(SRC_BUCKET, SRC_KEY, local_csv)
print("Downloaded:", local_csv)

# ---- 2) Load, prep, split ----
df = pd.read_csv(local_csv)
# encode label
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_csv = "train.csv"
test_csv  = "test.csv"
train_df.to_csv(train_csv, index=False, header=False)  # XGBoost expects no header
test_df.to_csv(test_csv, index=False, header=False)

# ---- 3) Upload to SageMaker bucket in this region ----
train_s3 = session.upload_data(train_csv, bucket=DST_BUCKET, key_prefix=f"{PREFIX}/train")
test_s3  = session.upload_data(test_csv,  bucket=DST_BUCKET, key_prefix=f"{PREFIX}/test")
print("Train S3:", train_s3)
print("Test  S3:", test_s3)

# ---- 4) Build TrainingInput channels ----
train_input = TrainingInput(train_s3,  content_type="text/csv")
test_input  = TrainingInput(test_s3,   content_type="text/csv")

# ---- 5) Get the XGBoost container for this region ----
# Pick a stable, low-cost CPU version of XGBoost
container_image = image_uris.retrieve(framework="xgboost", region=REGION, version="1.7-1")

# ---- 6) Try small/cheap instance types until one is allowed by your quotas ----
candidate_types = ["ml.m5.large", "ml.c5.large", "ml.m4.xlarge"]  # cheapest first

last_err = None
for itype in candidate_types:
    try:
        print(f"\nTrying training instance_type = {itype}")
        xgb = sagemaker.estimator.Estimator(
            image_uri=container_image,
            role=ROLE,
            instance_count=1,
            instance_type=itype,
            output_path=f"s3://{DST_BUCKET}/{PREFIX}/output",
            sagemaker_session=session,
        )
        # Modest hyperparams to keep job short/cheap
        xgb.set_hyperparameters(
            objective="binary:logistic",
            num_round=80,
            max_depth=5,
            eta=0.2,
            subsample=0.8,
            eval_metric="auc",
        )

        # Launch training (this automatically shuts down the training instance when done)
        xgb.fit({"train": train_input, "validation": test_input})
        print("\n✅ Training started successfully on", itype)
        last_err = None
        break
    except Exception as e:
        msg = str(e)
        last_err = e
        if "ResourceLimitExceeded" in msg or "limit" in msg.lower():
            print(f"⚠️  No quota for {itype}. Trying next option…")
        else:
            # Any other error should surface immediately
            raise

if last_err:
    raise last_err

# After the job completes, the trained model artifact is at:
# s3://{DST_BUCKET}/{PREFIX}/output/<training-job-name>/output/model.tar.gz


Region: us-east-1
Using SageMaker bucket: s3://sagemaker-us-east-1-191687650900/churn
Downloaded: churn.csv


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-09-17-21-23-54-960


Train S3: s3://sagemaker-us-east-1-191687650900/churn/train/train.csv
Test  S3: s3://sagemaker-us-east-1-191687650900/churn/test/test.csv

Trying training instance_type = ml.m5.large


ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-09-17-21-23-55-277
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


⚠️  No quota for ml.m5.large. Trying next option…

Trying training instance_type = ml.c5.large


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:81                                                                                   │
│                                                                                                  │
│   78 │   │   )                                                                                   │
│   79 │   │                                                                                       │
│   80 │   │   # Launch training (this automatically shuts down the training instance when done    │
│ ❱ 81 │   │   xgb.fit({"train": train_input, "validation": test_input})                           │
│   82 │   │   print("\n✅ Training started successfully on", itype)                               │
│   83 │   │   last_err = None                                                                     │
│   84 │   │   break                                                                               │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:168 in wrapper                                                                       │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:139 in wrapper                                                                       │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)              

In [3]:
from sagemaker import image_uris
import sagemaker

region = sagemaker.Session().boto_region_name
role   = sagemaker.get_execution_role()
container = image_uris.retrieve("xgboost", region, version="1.7-1")

from sagemaker.inputs import TrainingInput
train_input = TrainingInput(train_s3, content_type="text/csv")
test_input  = TrainingInput(test_s3,  content_type="text/csv")

itype = "ml.t3.medium"  # lowest-cost, valid training type

print(f"Trying training instance_type = {itype}")
xgb = sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type=itype,
    output_path=f"s3://{DST_BUCKET}/{PREFIX}/output",
    sagemaker_session=session,
    max_run=1800,  # cap at 30 min
    enable_sagemaker_metrics=True,
)

xgb.set_hyperparameters(
    objective="binary:logistic",
    num_round=120,
    max_depth=4,
    eta=0.2,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="auc",
)

xgb.fit({"train": train_input, "validation": test_input})


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-09-17-21-29-14-799


Trying training instance_type = ml.t3.medium


ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:36                                                                                   │
│                                                                                                  │
│   33 │   eval_metric="auc",                                                                      │
│   34 )                                                                                           │
│   35                                                                                             │
│ ❱ 36 xgb.fit({"train": train_input, "validation": test_input})                                   │
│   37                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:168 in wrapper                                                                       │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:139 in wrapper                                                                       │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                   

In [4]:
!pip install -q xgboost==1.7.6 joblib


In [6]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
import xgboost as xgb
import joblib, boto3, os
import sagemaker

# Load the same CSV you downloaded earlier
df = pd.read_csv("churn.csv")

# Label
y = df["Churn"].map({"Yes":1, "No":0}).astype(int)

# Features
X = df.drop(columns=["Churn"])

# Try to cast numeric-like fields (dataset uses CamelCase)
for col in ["tenure", "MonthlyCharges", "TotalCharges"]:
    if col in X.columns:
        X[col] = pd.to_numeric(X[col], errors="coerce")

# One-hot encode categoricals
X = pd.get_dummies(X, drop_first=True)

# Fill any NaNs
X = X.fillna(0)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train a small XGBoost model (fast/cheap)
clf = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=200,
    max_depth=4,
    learning_rate=0.2,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    n_jobs=2
)
clf.fit(X_train, y_train)

# Metrics
probs = clf.predict_proba(X_test)[:,1]
preds = (probs >= 0.5).astype(int)
auc = roc_auc_score(y_test, probs)
acc = accuracy_score(y_test, preds)
print(f"AUC: {auc:.3f}  |  Accuracy: {acc:.3f}")

# Save model locally
os.makedirs("model", exist_ok=True)
joblib.dump(clf, "model/model.joblib")

# Upload artifact to your SageMaker default bucket (same region)
session = sagemaker.Session()
dst_bucket = session.default_bucket()
prefix = "churn/output/local-model"
artifact_key = f"{prefix}/model.joblib"

boto3.client("s3").upload_file("model/model.joblib", dst_bucket, artifact_key)
print("Uploaded model to: s3://%s/%s" % (dst_bucket, artifact_key))


AUC: 0.824  |  Accuracy: 0.789
Uploaded model to: s3://sagemaker-us-east-1-191687650900/churn/output/local-model/model.joblib


In [7]:
# === Local XGBoost training (no SageMaker training quotas) ===

# 1) Install deps (safe to re-run; does nothing if already installed)
!pip install -q xgboost==1.7.6 joblib

# 2) Imports
import os, boto3, sagemaker, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
import xgboost as xgb
import joblib

# 3) Load data (uses the file you already downloaded earlier as churn.csv)
assert os.path.exists("churn.csv"), "churn.csv not found in working dir. Re-run your download cell."
df = pd.read_csv("churn.csv")

# 4) Label/feature prep
y = df["Churn"].map({"Yes":1, "No":0}).astype(int)
X = df.drop(columns=["Churn"])

# Cast numeric-like columns (dataset fields use CamelCase)
for col in ["tenure", "MonthlyCharges", "TotalCharges"]:
    if col in X.columns:
        X[col] = pd.to_numeric(X[col], errors="coerce")

# One-hot encode categoricals, fill NaNs
X = pd.get_dummies(X, drop_first=True).fillna(0)

# 5) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6) Train a small model (fast/cheap)
clf = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=200,
    max_depth=4,
    learning_rate=0.2,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    n_jobs=2,
)
clf.fit(X_train, y_train)

# 7) Metrics
probs = clf.predict_proba(X_test)[:, 1]
preds = (probs >= 0.5).astype(int)
auc = roc_auc_score(y_test, probs)
acc = accuracy_score(y_test, preds)
print(f"AUC: {auc:.3f}  |  Accuracy: {acc:.3f}")

# 8) Save model locally and upload to S3 for your portfolio
os.makedirs("model", exist_ok=True)
joblib.dump(clf, "model/model.joblib")

session = sagemaker.Session()
dst_bucket = session.default_bucket()
prefix = "churn/output/local-model"
artifact_key = f"{prefix}/model.joblib"
boto3.client("s3").upload_file("model/model.joblib", dst_bucket, artifact_key)
print("Uploaded model to: s3://%s/%s" % (dst_bucket, artifact_key))


AUC: 0.824  |  Accuracy: 0.789
Uploaded model to: s3://sagemaker-us-east-1-191687650900/churn/output/local-model/model.joblib


In [8]:
import boto3
boto3.client("s3").head_object(
    Bucket="sagemaker-us-east-1-191687650900",
    Key="churn/output/local-model/model.joblib"
)
print(" Model artifact is present in S3.")


 Model artifact is present in S3.
